# Junior embedded SW developer case

## Problems

### Problem 1
#### Description
Using the header functions defined in i2c.h, define the functions declared in
temp_sensor.h. To acquire a running program, you may mock the I2C bus (i.e. make
a dummy implementation of i2c.c where the function return values behave as if you
had a real system with a DSP310 connected to the I2C bus). You only need to mock
the parts relevant for reading the temperature, i.e. you do not need to simulate all
features of the sensor.

#### Proposed solution

Below are proposed implementations of the program. The sensor is emulated using an array of bytes. It is configured according to the data-sheet, following the descriptions given in the flow-chart on page 16. main.c is a test script, used to test if the code compiles and runs as expected.

##### i2c.h (as given)

```c
#ifndef I2C_H
#define I2C_H

#include <stdint.h>

/**
 * @brief Initialize I2C pins and registers
 * 
 * @return Zero on success, nonzero on error
 */
int i2c_init(void);

/**
 * @brief Write bytes to I2C bus
 * 
 * @param address Address of I2C device
 * @param data Buffer for data
 * @param len Number of bytes to write
 * @return Zero on success, nonzero on error
 */
int i2c_write(uint8_t address, uint8_t *data, uint8_t len);

/**
 * @brief Read bytes from I2C bus
 * 
 * @param address Address of I2C device
 * @param data Buffer for data
 * @param len Number of bytes to read
 * @return Zero on success, nonzero on error
 */
int i2c_read(uint8_t address, uint8_t *data, uint8_t len);

#endif


##### i2c.c

```c
#include "i2c.h"
#include <stdint.h>

/* Register definitions – these match what the sensor code expects. */
#define TMP_SENSOR_ADDR  0x77   // Sensor I2C address
#define TMP_CFG_REG      0x07   // Temperature configuration register
#define MEAS_CFG         0x08   // Measurement configuration register
#define CFG_REG          0x09   // FIFO/interrupt config register

// Temperature data registers (3 bytes)
#define TMP_B2_REG       0x03   // Temperature MSB
#define TMP_B1_REG       0x04
#define TMP_B0_REG       0x05

// Calibration registers (3 bytes)
#define TMP_C0_REG       0x10   // Calibration c0 (MSB)
#define TMP_C0C1_REG     0x11   // High nibble: lower bits of c0; Low nibble: upper bits of c1
#define TMP_C1_REG       0x12   // Calibration c1 (LSB)

static uint8_t sensor_memory[256];
static uint8_t sensor_pointer = 0;

int i2c_init(void) {
    /* Initialize calibration registers */
    sensor_memory[TMP_C0_REG]   = 0x00;
    sensor_memory[TMP_C0C1_REG] = 0x00;
    sensor_memory[TMP_C1_REG]   = 0x64;

    /* Initialize temperature data registers (simulate 25 degrees) */
    sensor_memory[TMP_B2_REG] = 0x20;
    sensor_memory[TMP_B1_REG] = 0x00;
    sensor_memory[TMP_B0_REG] = 0x00;
    
    return 0;
}

int i2c_write(uint8_t address, uint8_t *data, uint8_t len) {
    if (address != TMP_SENSOR_ADDR) {
        // For other addresses, do nothing.
        return 0;
    }
    if (len == 0) {
        return 0;
    }

    if (len == 1) {
        sensor_pointer = data[0];
    } else {
        // First byte is the register address; following bytes are data to write.
        sensor_pointer = data[0];
        for (uint8_t i = 1; i < len; i++) {
            sensor_memory[sensor_pointer++] = data[i];
        }
    }
    return 0;
}

int i2c_read(uint8_t address, uint8_t *data, uint8_t len) {
    if (address != TMP_SENSOR_ADDR) {
        // For any unknown device, return zeros.
        for (uint8_t i = 0; i < len; i++) {
            data[i] = 0;
        }
        return 0;
    }
    for (uint8_t i = 0; i < len; i++) {
        data[i] = sensor_memory[sensor_pointer++];
    }
    return 0;
}


##### temp_sensor.h (as given)

```c
#include "temp_sensor.h"
#include "i2c.h"
#include <stdint.h>

#define SENSOR_ADDR 0x48
#define TEMP_REG    0x00

int temp_sensor_init(void) {
    return i2c_init();
}

int temp_sensor_read(float *temp) {
    uint8_t reg = TEMP_REG;
    
    /* Write the temperature register pointer */
    if (i2c_write(SENSOR_ADDR, &reg, 1) != 0)
        return -1;
    
    uint8_t data[2];
    if (i2c_read(SENSOR_ADDR, data, 2) != 0)
        return -1;
    
    /* Combine the two bytes into an integer value (in hundredths of a degree) */
    int temp_int = (data[0] << 8) | data[1];
    
    /* Convert to a float representing degrees Celsius */
    *temp = temp_int / 100.0f;
    
    return 0;
}

##### temp_sensor.c

```c
#include "temp_sensor.h"
#include "i2c.h"

#define TMP_SENSOR_ADDR  0x77   // Default I2C address for DPS310 (SDO pin is not pulled down)
#define TMP_CFG_REG      0x07   // Temperature configuration register
#define MEAS_CFG         0x08   // Sensor operating mode and status
#define CFG_REG          0x09   // Interrupt and FIFO configuration

// Registers for temperature readings (3 bytes)
#define TMP_B2_REG       0x03   // MSB
#define TMP_B1_REG       0x04
#define TMP_B0_REG       0x05

// Calibration registers for temperature coefficients
#define TMP_C0_REG       0x10   // MSB
#define TMP_C0C1_REG     0x11
#define TMP_C1_REG       0x12

// Global calibration coefficients
static float c0 = 0.0f;
static float c1 = 0.0f;

// Temperature scaling factor (single precision)
static const int32_t kT = 524288;  // 2^19

// Sensor initialization
int temp_sensor_init(void)
{
    int status;

    /* 1. Read temperature calibration coefficients */
    uint8_t reg = TMP_C0_REG;
    
    status = i2c_write(TMP_SENSOR_ADDR, &reg, 1);
    if (status != 0) {
        return -1;  // Error setting calibration register address
    }

    uint8_t cal_data[3];
    status = i2c_read(TMP_SENSOR_ADDR, cal_data, 3);
    if (status != 0) {
        return -1;  // Error reading calibration data
    }

    /* Extract calibration registers */
    uint8_t reg0x10 = cal_data[0];
    uint8_t reg0x11 = cal_data[1];
    uint8_t reg0x12 = cal_data[2];

    // Compute c0 (12-bit, two's complement)
    int16_t raw_c0 = ((int16_t)reg0x10 << 4) | (reg0x11 >> 4);
    if (raw_c0 >= 2048) {
        raw_c0 -= 4096;
    }
    c0 = (float)raw_c0;

    // Compute c1 (12-bit, two's complement)
    int16_t raw_c1 = ((reg0x11 & 0x0F) << 8) | reg0x12;
    if (raw_c1 >= 2048) {
        raw_c1 -= 4096;
    }
    c1 = (float)raw_c1;

    /* 2. Set up sensor configuration for temperature measurement */
    uint8_t tmp_cfg[2];
    tmp_cfg[0] = TMP_CFG_REG;
    tmp_cfg[1] &= ~(1 << 7);      // TMP_EXT = 0 (internal sensor)
    tmp_cfg[1] &= ~(0b111 << 4);  // TMP_RATE = 000 (1 Hz)
    tmp_cfg[1] &= ~(0b1111);      // TMP_PRC = 0000 (single measurement)
    
    status = i2c_write(TMP_SENSOR_ADDR, tmp_cfg, 2);
    if (status != 0) {
        return -1;  // Error writing temperature configuration
    }

    /* 3. Set up interrupt and FIFO configuration */
    uint8_t cfg[2];
    cfg[0] = CFG_REG;
    cfg[1] = (1 << 1);  // Enable FIFO

    status = i2c_write(TMP_SENSOR_ADDR, cfg, 2);
    if (status != 0) {
        return -1;  // Error writing FIFO configuration
    }

    return 0;  // Success
}

// Read temperature
int temp_sensor_read(float *temp)
{
    int status;

    // Initiate a temperature measurement
    uint8_t meas_cfg[2] = {MEAS_CFG, 0x02}; // Starts temperature measurement

    status = i2c_write(TMP_SENSOR_ADDR, meas_cfg, 2);
    if (status != 0) {
        return -1;  // Error initiating temperature measurement
    }

    // Set the register pointer to the temperature data (MSB register)
    uint8_t reg = TMP_B2_REG;
    
    status = i2c_write(TMP_SENSOR_ADDR, &reg, 1);
    if (status != 0) {
        return -1;  // Error setting temperature data register address
    }

    // Read raw temperature data
    uint8_t data[3];
    
    status = i2c_read(TMP_SENSOR_ADDR, data, 3);
    if (status != 0) {
        return -1;  // Error reading temperature data
    }

    // Combine data bytes
    uint32_t raw24 = ((uint32_t)data[0] << 16) | ((uint32_t)data[1] << 8) | data[2];
    int32_t tmp_raw = raw24 >> 4;  // Discard the lower 4 fractional bits

    if (tmp_raw & (1 << 19)) {
        tmp_raw -= (1 << 20);
    }

    // Scale the raw temperature data
    float tmp_raw_sc = (float)tmp_raw/kT;

    // Calculate the compensated temperature
    float tmp_comp = (c0*0.5f) + (c1*tmp_raw_sc);

    // Return calculated temperature
    *temp = tmp_comp;

    return 0;  // Success
}


##### main.c

```c
#include <stdio.h>
#include "temp_sensor.h"
#include "i2c.h"

int main(void) {
    int status;
    float temperature;

    /* Initialize the I2C interface. */
    status = i2c_init();
    if (status != 0) {
        printf("I2C initialization failed!\n");
        return 1;
    }

    /* Initialize the temperature sensor. */
    status = temp_sensor_init();
    if (status != 0) {
        printf("Temperature sensor initialization failed!\n");
        return 1;
    }

    /* Read the temperature from the sensor. */
    status = temp_sensor_read(&temperature);
    if (status != 0) {
        printf("Failed to read temperature!\n");
        return 1;
    }

    /* Print the measured temperature. */
    printf("Measured temperature: %.2f °C\n", temperature);

    return 0;
}



##### Running main.c

In [1]:
gcc main.c temp_sensor.c i2c.c -o test_sensor
./test_sensor

Measured temperature: 25.00 °C


### Problem 2
#### Description
You come in to work and see in the commit log that your colleague developed a driver for another sensor. They pushed their changes to the master branch. The sensor happens to use the same I2C bus as your temperature sensor. You inspect the new code files your colleague made, and copy them to the folder you are working in. You rerun the tests you did earlier, but you notice that the data read back from the temperature sensor sometimes has nonsensical values.

<strong>a.</strong> &nbsp; What might be the reason behind the nonsensical values? How could this problem be mitigated?

<details>
  <summary><strong>Answer:</strong></summary>

  The sensors might have overlapping addresses on the i2c bus. This could be mitigated by either 
  1) choosing sensors with different addresses,
  2) configure one of the sensors with a different address (such as configuring the SDO pin on this particular sensor), or
  3) use an i2c mux.

</details>

<strong> b.</strong> &nbsp; With version control in mind, discuss the negative implications of the workflow that was used in this example.
<details>
  <summary><strong>Answer:</strong></summary>
  The other colleague pushed their changes to the master branch; they should probably have been working on a branch. Furthermore, the code was not run through proper checks, neither review nor system tests. There should have been clearer communication or better proceedures implemented to prevent these types of mistakes.

</details>

### Problem 3
#### Description
You compiled your project with arm-none-eabi-gcc and flashed it to the MCU, and
everything seems to be working nicely. Well done! You tell your colleague about
your success, and one hour later, they show up at your desk saying they had some
problems. Your colleague says the project “compiles fine with gcc”, but you get an
error message when attempting to flash their compiled program to the MCU. What
might be wrong?

<details>
  <summary><strong>Answer:</strong></summary>
  They used gcc instead of arm-none-eabi-gcc, thus they did not compile for the architecture that they were flashing to. gcc only compiles for their computer's architecture.

</details>


### Problem 4
#### Description
Your manager comes by your desk saying the requirements for the accuracy of the
temperature measurements have been updated. The customer wants temperature
measurements with an accuracy of +/- 0.2 degrees C. Can this be achieved with
DSP310? If yes, how? If not, why?

<details>
  <summary><strong>Answer:</strong></summary>
  The DPS310 has a temperature accuracy of +/- 0.5. The precision can be increased through oversampling (a feature that is built into the DPS310), but if the measurement variance is due to bias in the measurements, and not only due to zero-mean noise, it cannot be reduced by increasing the precision/oversampling. The accuracy rating probably stems from the calibration process, and increasing accuracy would probably require better/new calibration, although this might be hard to do better than the manufacturer.

</details>


### Problem 5
#### Description
Your team is developing a new circuit board using the DSP310 to read pressure and
temperature. The board houses an ARM-based MCU which supports the following
GPIO pad configurations for the I2C SDA and SCL lines:


&nbsp; &nbsp; a. &nbsp; Open drain\
&nbsp; &nbsp; b. &nbsp; Pull-up\
&nbsp; &nbsp; c. &nbsp; Pull-down

After flashing a simple test program that attempts to get a temperature reading from the sensor, you notice that the SDA and SCL lines are always 0V. What pad configuration could solve this problem, and why?

<details>
  <summary><strong>Answer:</strong></summary>
    You need to configure the IO pads for the SDA and the SCL lines as pull-up, as decives on the I2C bus only pulls the lines down, but cannot pull them up.
</details>
